In [107]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from tqdm import tqdm
import re
import openai
import time
openai.api_key = "insert-openai-API"

In [2]:
hoteles = pd.read_csv('hoteles-mad.csv')

In [3]:
hoteles.isna().sum()

_id              0
descripcion      0
link           148
nombre           0
precio           0
puntuacion       0
serv_hab         0
tipo_hab         0
ubicacion      788
dtype: int64

In [4]:
hoteles.drop(columns = ['_id'], inplace = True)

In [5]:
check = hoteles[['descripcion', 'link','precio','puntuacion','serv_hab','tipo_hab','ubicacion']]

In [6]:
dups = check.duplicated()

In [7]:
dup_index=check.index[dups]

In [8]:
hoteles = hoteles.drop(index=dup_index)

In [9]:
hoteles = hoteles.loc[~hoteles.ubicacion.isna()]

In [33]:
from410 = list(hoteles.ubicacion[410::])

['Cruz 26, 3 Derecha, 28012 Madrid España',
 'Calle Carretas 3 2nd, 4th Floors, 28012 Madrid España',
 'Conde de Romanones 9, 2 Piso, 28012 Madrid España',
 'Calle Espejo 2 Piso 2, 28013 Madrid España',
 'Calle Atocha 43, 28012 Madrid España',
 'Calle Cava de San Miguel 8, 28005 Madrid España',
 'Calle Salud 13, 28013 Madrid España',
 'Calle Esparteros 12, 28012 Madrid España',
 'Calle Zorrilla 9, 28014 Madrid España',
 'Avenida Logrono 333 Barajas, 28042 Madrid España',
 'Calle de Don Ramón de la Cruz 70, 28006 Madrid España',
 'Calle Victoria 1, 28012 Madrid España',
 'Calle Ventura de La Vega 5, 28014 Madrid España',
 'Calle Juan de Olias 15 metro ESTRECHO, linea 1, 28020 Madrid España',
 'Calle Chinchilla 1, 28013 Madrid España',
 'Calle Vascos 27, 28040 Madrid España',
 'Calle Factor 5, 28013 Madrid España',
 'Gran Via 15 - planta 6, 28013 Madrid España',
 'Calle Batalla de Torrijos, 26, 28025 Madrid España',
 'Calle Fuencarral 95, 28004 Madrid España',
 'Calle del Leon 13 Planta 

In [34]:
system_content='''
You are a data wrangler. The input will be an address. 

The target is to extract the address corrected and simplified.
Do not write anything else than the street name, number, zip code and city. 
In case of the address being already simplified, leave it as it is. In case of not being able to cimplify it, leave it as it is.

Example:
INPUT : 'Mejia Lequerica 10. Mercado Barceló. Madrid Mercado Barceló - Segunda Planta Interior. Referencia: Acceso Frente Al Hotel Eurostars., 28004, Madrid'
OUTPUT : 'Calle de Mejía Lequerica, 10, 28004, Madrid'

Example 2:
INPUT: 'C / Gran Vía, 55 Oven, Steakburger y Pura Kaffeina, 28013, Madrid'
OUTPUT: 'Calle Gran Vía, 55, 28013, Madrid'

Example 3:
INPUT: 'Calle Gran Via 80 Junto a Plaza de España, 28013 Madrid España'
OUTPUT: 'Calle Gran Vía, 80, 28013, Madrid'

'''
#%%time
direccion = []

for e in tqdm(from410):
    response=openai.ChatCompletion.create(model='gpt-3.5-turbo', 
                                          messages=[{'role': 'system', 'content': system_content},
                                                    {'role': 'user', 'content': e}])

    direccion.append(response['choices'][0]['message']['content'])
    time.sleep(3)

100%|████████████████████████████████████████████████████████████████████████████████| 435/435 [31:07<00:00,  4.29s/it]


In [23]:
hoteles.reset_index(drop=True, inplace=True)

In [43]:
index = list(range(0,len(direccion)))
len(index)

845

In [45]:
for i in range(len(index)):
    hoteles.at[index[i], 'ubicacion'] = direccion[i]

In [48]:
lat = []
long = []

geolocator = Nominatim(user_agent="my_app")

for e in tqdm(hoteles.ubicacion):
    try:
        loc = geolocator.geocode(e)
        lat.append(loc.latitude)
        long.append(loc.longitude)
    except:
        lat.append('No disponible')
        long.append('No disponible')

100%|████████████████████████████████████████████████████████████████████████████████| 845/845 [07:05<00:00,  1.98it/s]


In [49]:
hoteles['lat'] = lat
hoteles['long'] = long

In [54]:
hoteles.drop(hoteles[hoteles.lat == 'No disponible'].index, inplace = True)

In [58]:
hoteles.to_csv('../hoteles/hoteles-mad-ubi.csv', index = False)

In [90]:
servs = []
for e in hoteles.serv_hab:
    servs.append(e.replace('"','').replace('[','').replace(']','').split(','))

In [93]:
hoteles['serv_hab'] = servs

In [97]:
rooms = []

for e in hoteles.tipo_hab:
    rooms.append(e.replace('"','').replace('[','').replace(']','').split(','))

In [99]:
hoteles['tipo_hab'] = rooms

In [102]:
dummies = hoteles['serv_hab'].apply(lambda x: pd.Series([1]*len(x), index=x)).fillna(0, downcast='infer')

In [103]:
dummies2 = hoteles['tipo_hab'].apply(lambda x: pd.Series([1]*len(x), index=x)).fillna(0, downcast='infer')

In [104]:
hotel = pd.concat([hoteles, dummies, dummies2], axis = 1)

In [106]:
hotel.to_csv('../hoteles/hoteles-mad-dummies.csv', index = False)

In [130]:
hotel2 = pd.read_csv('../hoteles/hoteles-mad-dummies.csv')

In [133]:
hotel2.drop(columns = ['serv_hab','tipo_hab','link'], inplace = True)

In [136]:
precio = []
for e in hotel2.precio:
    if e == 'No disponible':
        precio.append(0)
    else:
        e = e.replace('.','')
        precio.append(int(e[:-2]))

In [137]:
hotel2['precio'] = precio 

In [142]:
punt = []
for e in hotel2.puntuacion:
    punt.append(e.replace(',','.'))

In [144]:
hotel2['puntuacion'] = punt

In [149]:
hotel2 = hotel2.reset_index(drop = True)

In [150]:
hotel2.to_csv('../hoteles/hoteles-mad-dummies.csv', index = False)